# 🤖 Workshop: สร้าง AI ตอบคำถามจากเอกสาร ด้วย RAG Framework
### ดร. CJ - Walailak University
เวิร์กชอปนี้ออกแบบสำหรับนักเรียนมัธยมปลาย เพื่อให้เข้าใจแนวคิด RAG (Retrieval-Augmented Generation) และลองสร้างระบบ AI ตอบคำถามจากเอกสารได้ด้วยตัวเอง

In [ ]:
!pip install sentence-transformers faiss-cpu openai -q
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI
import numpy as np

print('✅ เครื่องมือพร้อมใช้งานแล้ว!')

## 🧩 ส่วนที่ 1: สร้างเอกสารตัวอย่าง

In [ ]:
text = '''
สมองของมนุษย์ประกอบด้วยเซลล์ประสาทนับพันล้านเซลล์
ซึ่งส่งสัญญาณไฟฟ้าเพื่อให้เราคิด วิเคราะห์ และจดจำสิ่งต่าง ๆ ได้
ระบบประสาททำงานร่วมกับประสาทสัมผัส เช่น การมองเห็นและการได้ยิน
ทำให้มนุษย์สามารถตอบสนองต่อสิ่งรอบตัวได้อย่างรวดเร็ว
'''
print('ตัวอย่างเอกสาร:')
print(text)

## ⚙️ ส่วนที่ 2: แบ่งเอกสารเป็นชิ้นเล็ก ๆ (Chunking)

In [ ]:
chunk_size = 80
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
print(f'
จำนวนชิ้นข้อมูล (chunks): {len(chunks)}
')
for i, c in enumerate(chunks):
    print(f'Chunk {i+1}: {c}
')

## 🧠 ส่วนที่ 3: ใช้ LLM สร้างสรุป (จำลอง Context Generation)

In [ ]:
def fake_summary(chunk):
    return f'สรุป: {chunk[:30]}...'

summaries = [fake_summary(c) for c in chunks]
for s in summaries:
    print(s)

## 🔢 ส่วนที่ 4: แปลงข้อความเป็นเวกเตอร์ (Embedding)

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(summaries)
print(f'
Embedding ตัวอย่าง: {embeddings[0][:10]}')

## 🔍 ส่วนที่ 5: สร้างระบบค้นหา (Retriever)

In [ ]:
query = 'มนุษย์คิดได้อย่างไร'
query_vec = model.encode([query])
scores = cosine_similarity(query_vec, embeddings)
for i, score in enumerate(scores[0]):
    print(f'Chunk {i+1} | คะแนนความใกล้เคียง: {score:.3f}')

best_idx = np.argmax(scores)
print('
🔍 ข้อมูลที่ใกล้เคียงที่สุดคือ:')
print(chunks[best_idx])

## 💬 ส่วนที่ 6: ให้ AI สร้างคำตอบ (จำลอง)

In [ ]:
def fake_answer(context, question):
    return f'จากข้อมูล: {context[:50]}...
คำตอบ: มนุษย์สามารถคิดได้เพราะสมองมีเซลล์ประสาทจำนวนมากที่เชื่อมโยงกัน'

context = chunks[best_idx]
answer = fake_answer(context, query)
print('
🤖 คำตอบของ AI:
', answer)

## 🎮 ส่วนที่ 7: ให้นักเรียนลองทำเอง!

In [ ]:
user_q = input('พิมพ์คำถามของคุณ: ')
user_vec = model.encode([user_q])
scores = cosine_similarity(user_vec, embeddings)
best_idx = np.argmax(scores)

print('
🔍 ข้อมูลที่เกี่ยวข้อง:')
print(chunks[best_idx])

print('
🤖 คำตอบจำลองของ AI:')
print(fake_answer(chunks[best_idx], user_q))